# 前処理
- データの取得
- データの整形
    - 前処理
    - 形態素解析
    - 単語数の絞り込み

# 0. Prepare

In [ ]:
from time import time
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')
import pickle
import numpy as np
from tqdm import tqdm
import re
import os
from collections import Counter

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install emoji
!pip install neologdn
import emoji
import neologdn

# 形態素解析
# https://qiita.com/jun40vn/items/78e33e29dce3d50c2df1

# 形態素分析ライブラリーMeCab と 辞書(mecab-ipadic-NEologd)のインストール 
!apt-get -q -y install sudo file mecab libmecab-dev mecab-ipadic-utf8 git curl python-mecab > /dev/null
!git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git > /dev/null 
!echo yes | mecab-ipadic-neologd/bin/install-mecab-ipadic-neologd -n > /dev/null 2>&1
!pip install mecab-python3 > /dev/null

# シンボリックリンクによるエラー回避
!ln -s /etc/mecabrc /usr/local/etc/mecabrc

import MeCab

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
E: Unable to locate package python-mecab
fatal: destination path 'mecab-ipadic-neologd' already exists and is not an empty directory.
ln: failed to create symbolic link '/usr/local/etc/mecabrc': File exists


## 1. データの取得

In [ ]:
path = 'input/'

In [ ]:
path_list = sorted(os.listdir(path))
for i, p in enumerate(path_list):
    print(i, p)

0 mid_drive_20100107-20211231.csv
1 old_drive_20100101-20160501.txt
2 old_drive_20160430-20190520.txt
3 old_drive_20190519-20211101.txt
4 old_drive_20211001-20220101.txt
5 yng_drive_20100102-20211101.txt
6 yng_drive_20211001-20220101.txt


In [ ]:
%%time
# yng
df_yng = pd.concat([pickle.load(open(path+path_list[x], 'rb'))[['created_at', 'text']] for x in [5,6]]).reset_index(drop=True)
df_yng

CPU times: user 1.07 s, sys: 421 ms, total: 1.49 s
Wall time: 1.53 s


,created_at,text
0,2010-01-02 11:10:04+09:00,若者達は、日本から脱出したほうがいいかもしれない。経歴になんも問題もない奴は、たとえトラック...
1,2010-01-02 20:41:58+09:00,運転してみてわかったのは、横断歩道のスタート地点でないところから道路を横切ることが自分の想像...
2,2010-01-03 19:17:22+09:00,「このような、危険な運転をする若者が、後を絶たない」（CV旦那
3,2010-01-04 02:26:10+09:00,今日夕ご飯王将行ったー名前を書こうとしたらババァに横入りされた。本当オバサンって『最近の若者...
4,2010-01-04 19:44:56+09:00,近鉄線の運転手席つき連結部分は若者専用ボックスシート http://twitgoo.com/...
...,...,...
316129,2022-01-01 14:21:01+09:00,１６歳～３９歳の通学・通勤者が対象の若年層コロナワクチン❕ 大行列で、先着順じゃなく抽選で...
316130,2022-01-01 14:36:44+09:00,@oideiiko 若さisgodを佐伯のような若者にこそ俺は伝えたかったんだよな 金...
316131,2022-01-01 15:17:53+09:00,@hazak0515 ちーさん、明けましておめでとうございます🎍🐯🎍🐯 ちーさんも健康で...
316132,2022-01-01 17:37:40+09:00,文系「死ぬ気で研究開発せんかぁ！」 理系「給料上げて」 文系「足りぬ足りぬは努力が足りぬ（ｷ...


In [ ]:
%%time
# mid
df_mid = pd.read_csv(path+path_list[0], engine='python')[['created_at', 'text']]
df_mid

CPU times: user 126 ms, sys: 12.5 ms, total: 138 ms
Wall time: 138 ms


,created_at,text
0,2010-01-07 13:35:13+09:00,ドライバーのオッサン、Bollywoodフィルムにでてきそうなハンサム中年。GSで給油中にエ...
1,2010-01-09 13:36:40+09:00,高次脳機能障害のある中年男性。かかりつけ医は眼科。運転免許、復職のための診断書を書く医者を行...
2,2010-01-10 01:14:12+09:00,"[tbr] ""かつては「リタイヤしたら田舎で悠々自適」というのが中高年にとって理想の人生では..."
3,2010-01-10 10:17:44+09:00,守備範囲は干支一周上まで！ RT @krowed: 中年キラーｗ RT @heidi22...
4,2010-01-11 08:48:02+09:00,"へー！ ""かつては「リタイヤしたら田舎で悠々自適」というのが中高年にとって理想の人生ではあっ..."
...,...,...
25162,2021-12-31 12:35:22+09:00,歳相応の常識が身につけられてない中年以降はそれなりに多い 圏央道のトンネルで蛇行運転してた...
25163,2021-12-31 12:55:21+09:00,@pivomania お年寄りや杖ついている人や赤子連れを意識するのは良いと思うが、若者や...
25164,2021-12-31 14:23:54+09:00,中年ペーパードライバーは要練習ということで群馬までドライブです。 https://t.co/...
25165,2021-12-31 17:36:43+09:00,昭和の高校生が信じていた伝説を中年になって実行する足りなさ。 警察官の目の前でビール 「飲...


In [ ]:
%%time
# old
df_old = pd.concat([pickle.load(open(path+path_list[x], 'rb'))[['created_at', 'text']] for x in [1,2,3,4]]).reset_index(drop=True)
df_old

CPU times: user 8.25 s, sys: 3.49 s, total: 11.7 s
Wall time: 13.3 s


,created_at,text
0,2010-01-01 15:18:36+09:00,住基ネット離脱、名古屋市長？という記事がYahooで掲載されているニュースで流れていますが。...
1,2010-01-01 16:51:05+09:00,うちの親も含めて、高齢者が運転する車が増えてるだろうなぁ。いままでとは違う世界がもうとっくに...
2,2010-01-01 22:16:05+09:00,運転免許保有者、約８千万人の中で60歳以上の高齢ドライバーの割合は23％と急増。消費市場にも...
3,2010-01-02 18:25:30+09:00,@chacha_oyaji 今、昨年の交通事故死亡者が東京都より埼玉県のほうが多いって報道...
4,2010-01-02 20:39:24+09:00,高齢者の事故が多いのは、高齢者が本来お互いが気をつけなくてはいけない部分をドライバーに委ね過...
...,...,...
2297720,2022-01-01 17:32:45+09:00,【新春】さっそくコンビニに車突っ込む サガ この車を運転していたのは佐賀市に住む70代の男...
2297721,2022-01-01 17:42:40+09:00,@HATAKEYAMANOUEN 除雪車運転手さんの不足と高齢化問題は、よく聞いております...
2297722,2022-01-01 17:44:29+09:00,かつて「俺は衰えを感じたら自主的に免許を返納するわ」と言ってた人間が今こうして運転事故を起こ...
2297723,2022-01-01 17:48:54+09:00,一定の違反歴がある高齢者ドライバーに実車試験を義務付け、閣議決定 #ldnews https...


In [ ]:
dfs = [df_yng.copy(), df_mid.copy(), df_old.copy()]

## 2. データの整形

### 2.1 重複削除

In [ ]:
%%time
# 重複テキストの削除
for i, df in enumerate(dfs):
    print(f'before: {len(df)}')
    df = df.drop_duplicates(subset='text').reset_index(drop=True)
    dfs[i] = df
    print(f'after: {len(df)}')
    print()

before: 316134
after: 102042

before: 25167
after: 25146

before: 2297725
after: 814801

CPU times: user 3.33 s, sys: 432 ms, total: 3.76 s
Wall time: 3.7 s


### 2.2 前処理

In [ ]:
# 前処理
    
def text_preprocess(doc):

    # 正規表現パターンを正規表現オブジェクトにコンパイル
    re1 = re.compile(r"@([a-zA-Z_0-9:]+):")
    re2 = re.compile(r"#(\w+)")
    re3 = re.compile(r"(https?|ftp)(:\/\/[-_\.!~*\'()a-zA-Z0-9;\/?:\@&=\+\$,%#]+)")
    re4 = re.compile(r"@([a-zA-Z_:0-9]+)")
    for r in [re1, re2, re3, re4]:
        doc = r.sub('', doc)
    
    # 絵文字削除
    doc = emoji.replace_emoji(doc)

    # 記号削除
    pattern = '[!"#$%&\'\\\\()*+,-./:;<=>?@[\\]^_`{|}~「」〔〕“”◇ᴗ●↓→♪★⊂⊃※△□◎〈〉『』【】＆＊・（）＄＃＠？！｀＋￥]' #。、    
    doc =  re.sub(pattern, ' ', doc)

    # 正規化
    doc = neologdn.normalize(doc)    
    doc = doc.replace('RT', '').replace('QT', "")

    # 小文字化
    doc = doc.lower()    

    return doc

def calc_lens(texts):
    return round(pd.Series([len(text) for text in texts]).describe(),2).to_list()

def df_shaping(df, start='2010-01-01', end='2021-12-31'):
    df = df.dropna().reset_index(drop=True)

    df_describe = pd.DataFrame(columns=['count', 'mean', 'std', 'min', '25%', '50%', '75%', 'max'])

    name = 'オリジナル'
    df_describe.loc[name] = calc_lens(df['text'])
    print(f'{name}: {len(df):,}')

    name = '日付処理後'
    df['created_at'] = pd.to_datetime(df['created_at'])
    df['time'] = df['created_at'].dt.tz_localize(None).map(lambda x: str(x).split(' ')[0])
    df = df.drop('created_at', axis=1)    
    df = df[df['time']>=start]
    df = df[df['time']<=end]
    df_describe.loc[name] = calc_lens(df['text'])
    print(f'{name}: {len(df):,}')


    name = '前処理後'
    df['preprocessed'] = df['text'].map(text_preprocess)
    df_describe.loc[name] = calc_lens(df['preprocessed'])
    print(f'{name}: {len(df):,}')  
    

    name = '重複削除後'
    df = df.drop_duplicates(subset='preprocessed').reset_index(drop=True)
    df_describe.loc[name] = calc_lens(df['preprocessed'])
    print(f'{name}: {len(df):,}')

    display(df_describe)
    return df, df_describe

In [ ]:
%%time
for i, df in enumerate(dfs):
    df, _ = df_shaping(df)
    dfs[i] = df
    print('-'*50)

オリジナル: 102,042
日付処理後: 102,027
前処理後: 102,027
重複削除後: 91,023


,count,mean,std,min,25%,50%,75%,max
オリジナル,102042.0,105.87,36.32,5.0,77.0,111.0,138.0,830.0
日付処理後,102027.0,105.86,36.32,5.0,77.0,111.0,138.0,830.0
前処理後,102027.0,85.65,33.77,2.0,57.0,88.0,115.0,180.0
重複削除後,91023.0,88.86,33.06,2.0,62.0,93.0,117.0,180.0


--------------------------------------------------
オリジナル: 25,145
日付処理後: 25,145
前処理後: 25,145
重複削除後: 22,933


,count,mean,std,min,25%,50%,75%,max
オリジナル,25145.0,105.23,38.28,5.0,75.0,112.0,138.0,413.0
日付処理後,25145.0,105.23,38.28,5.0,75.0,112.0,138.0,413.0
前処理後,25145.0,86.50,34.76,0.0,58.0,90.0,117.0,157.0
重複削除後,22933.0,88.22,34.41,0.0,60.0,92.0,118.0,157.0


--------------------------------------------------
オリジナル: 814,801
日付処理後: 814,714
前処理後: 814,714
重複削除後: 707,109


,count,mean,std,min,25%,50%,75%,max
オリジナル,814801.0,96.52,39.46,5.0,65.0,97.0,134.0,880.0
日付処理後,814714.0,96.52,39.46,5.0,65.0,97.0,134.0,880.0
前処理後,814714.0,75.96,36.49,0.0,44.0,74.0,108.0,272.0
重複削除後,707109.0,79.88,36.26,0.0,49.0,81.0,112.0,272.0


--------------------------------------------------
CPU times: user 1min 37s, sys: 843 ms, total: 1min 37s
Wall time: 1min 37s


In [ ]:
dfs_ = dfs.copy()
# dfs = dfs_.copy()

### 2.3 形態素解析

#### user_dict

In [ ]:
# #mecabのインストール
# !cd ../usr/local/lib
# !sudo apt install mecab
# !sudo apt install libmecab-dev
# !sudo apt install mecab-ipadic-utf8

# #ユーザ辞書用のフォルダ作成
# !mkdir ../var/lib/mecab/dic/userdic

# #日化辞、ＪＳＴのdicをダウンロード。上で作ったユーザ辞書用のフォルダに置く。
# !curl -o ../var/lib/mecab/dic/userdic/Nikkaji.dic ~~(適当な場所)/Nikkaji.dic
# !curl -o ../var/lib/mecab/dic/userdic/Thesaurus2015.dic ~~(適当な場所)/Thesaurus2015.dic

# #追加のユーザ辞書作成。上と同じフォルダに追加
# import pandas as pd    
# df_userdic = pd.read_csv('/content/drive/MyDrive/03_学校/高齢者ドライバー/github/data/drive_user_dict2.csv') 
# #df_userdicはIPA品詞体系と同じ並び
# df_userdic.to_csv('/content/drive/MyDrive/03_学校/高齢者ドライバー/github/data/drive_user_dict2.csv',header=None,index=False,encoding='utf-8')

# !/usr/lib/mecab/mecab-dict-index \
# -d /var/lib/mecab/dic/debian \
# -u /content/userdic.dic \
# -f utf-8 \
# -t utf-8 /content/userdic.csv
# !mv -f /content/userdic.dic /var/lib/mecab/dic/userdic/userdic2.dic


# #mecabrcの置き換え:Nikkaji.dic、Thesaurus2015.dic、userdic2.dicを
# #ユーザ辞書として指定したmecabrcに置き換え。
# #colaboratoryだとviなど使えずmecarcを直接編集できないので、
# #一回他でmecabrcのuserdic=の部分を下記のような感じで書き換えておく。
# # userdic = /var/lib/mecab/dic/userdic/Nikkaji.dic,/var/lib/mecab/dic/userdic/Thesaurus2015.dic,/var/lib/mecab/dic/userdic/userdic2.dic

# !curl -o /etc/mecabrc  ~~適当な場所/mecabrc
# !pip install mecab-python3==0.7
# import MeCab

In [ ]:
## 辞書作成
# 【ユーザー辞書(確認)】
cols = '表層形,左文脈ID,右文脈ID,生起コスト,品詞,品詞細分類１,品詞細分類２,品詞細分類３,活用型,活用形,原形,読み,発音'.split(',')
# cols = '表層形,生起コスト,品詞,品詞細分類１,品詞細分類２,品詞細分類３,活用型,活用形,原形,読み,発音'.split(',')

path = 'data/drive_user_dict.csv'
display(pd.read_csv(path, names=cols))

# 【辞書の登録】
! /usr/lib/mecab/mecab-dict-index \
    -m mecab-ipadic-2.7.0-20070801.model.utf8 \
    -d /var/lib/mecab/dic/debian -t utf-8 \
    -u terminology.dic \
    'data/drive_user_dict.csv' -f utf-8


,表層形,左文脈ID,右文脈ID,生起コスト,品詞,品詞細分類１,品詞細分類２,品詞細分類３,活用型,活用形,原形,読み,発音
0,twitter,＊,＊,0,辞書登録,辞書登録,＊,＊,＊,＊,twitter,＊,＊
1,出来る,＊,＊,0,辞書登録,辞書登録,＊,＊,＊,＊,できる,＊,＊
2,asahi,＊,＊,0,辞書登録,辞書登録,＊,＊,＊,＊,朝日,＊,＊
3,クルマ,＊,＊,0,辞書登録,辞書登録,＊,＊,＊,＊,自動車,＊,＊
4,歩行者,＊,＊,0,辞書登録,辞書登録,＊,＊,＊,＊,歩行者,＊,＊
...,...,...,...,...,...,...,...,...,...,...,...,...,...
118,2018,＊,＊,18,辞書登録,辞書登録,＊,＊,＊,＊,2018,＊,＊
119,2019,＊,＊,19,辞書登録,辞書登録,＊,＊,＊,＊,2019,＊,＊
120,2020,＊,＊,20,辞書登録,辞書登録,＊,＊,＊,＊,2020,＊,＊
121,2021,＊,＊,21,辞書登録,辞書登録,＊,＊,＊,＊,2021,＊,＊


/var/lib/mecab/dic/debian/pos-id.def is not found. minimum setting is used
reading /content/drive/MyDrive/03_学校/高齢者ドライバー/github/data/drive_user_dict.csv ... 123
emitting double-array: 100% |###########################################| 

done!


#### stopwords

In [ ]:
stopwords = ['あそこ',
#  'あたり',
 'あちら',
 'あっち',
 'あと',
 'あな',
#  'あなた',
 'あれ',
 'いくつ',
 'いつ',
 'いま',
 'いや',
 'いろいろ',
 'うち',
 'おおまか',
#  'おまえ',
#  'おれ',
 'がい',
 'かく',
 'かたち',
 'かやの',
 'から',
 'がら',
#  'きた',
 'くせ',
 'ここ',
 'こっち',
 'こと',
 'ごと',
 'こちら',
 'ごっちゃ',
 'これ',
 'これら',
 'ごろ',
#  'さまざま',
 'さらい',
 'さん',
 'しかた',
 'しよう',
 'すか',
 'ずつ',
 'すね',
#  'すべて',
#  'ぜんぶ',
 'そう',
 'そこ',
 'そちら',
 'そっち',
 'そで',
 'それ',
#  'それぞれ',
 'それなり',
#  'たくさん',
 'たち',
 'たび',
 'ため',
#  'だめ',
 'ちゃ',
 'ちゃん',
 'てん',
 'とおり',
 'とき',
 'どこ',
#  'どこか',
 'ところ',
 'どちら',
 'どっか',
 'どっち',
 'どれ',
 'なか',
#  'なかば',
 'なに',
 'など',
 'なん',
#  'はじめ',
 'はず',
 'はるか',
 'ひと',
 'ひとつ',
 'ふく',
 'ぶり',
 'べつ',
 'へん',
 'ぺん',
 'ほう',
 'ほか',
 'まさ',
 'まし',
 'まとも',
 'まま',
 'みたい',
 'みつ',
#  'みなさん',
#  'みんな',
 'もと',
 'もの',
 'もん',
 'やつ',
 'よう',
 'よそ',
 'わけ',
#  'わたし',
 'ハイ',
#  '彼女',
#  '下記',
#  '上記',
#  '時間',
#  '今回',
#  '前回',
#  '場合',
#  '一つ',
 '年生',
#  '自分',
 'ヶ所',
 'ヵ所',
 'カ所',
 '箇所',
 'ヶ月',
 'ヵ月',
 'カ月',
 '箇月',
#  '名前',
#  '本当',
#  '確か',
#  '時点',
#  '全部',
#  '関係',
#  '近く',
#  '方法',
#  '我々',
#  '違い',
#  '多く',
#  '扱い',
#  '新た',
#  'その後',
 '半ば',
#  '結局',
#  '様々',
#  '以前',
#  '以後',
#  '以降',
#  '未満',
#  '以上',
#  '以下',
 '幾つ',
#  '毎日',
#  '自体',
 '向こう',
 '何人',
#  '手段',
#  '同じ',
#  '感じ'
 ]


In [ ]:
import requests
url = "http://svn.sourceforge.jp/svnroot/slothlib/CSharp/Version1/SlothLib/NLP/Filter/StopWord/word/Japanese.txt"
r = requests.get(url)
tmp = r.text.split('\r\n')
stopwords = []
for i in range(len(tmp)):
    if len(tmp[i]) < 2:
        continue
    stopwords.append(tmp[i])

#### main

In [ ]:
nayose = {'くる':'来る', 'いう':'言う', 'いい':'良い', 'よい':'良い', 'いく':'行く', 'はねる':'跳ねる', 'ない':'無い', 'すぎる':'過ぎる', 'ほしい':'欲しい', 'ウインカ':'ウインカー', 'みる':'見る', 'くださる':'下さる', 'こわい':'怖い'}


stops = """
、運転、ドライバー、できる、られる、しれる、おる、しまう、やすい、ところ、わかる、さん、とき、
、そこ、そう、づらい、せる、わけ、どころ、もらう、いける、つく、でる、くれる、ちゃん、あれ、なに、
、こっち、gt、at、とこ、こちら、さっき、とく、lt、える、だし、ちる、たつ、たま、あく、こいつ、コレ、www、ww、スマ、
、ので、思う、より、きる、あり、出る、the、使う、いる、ある、する、こと、ため、なる、れる、って、その、
、*、もの、よう、どこ、どれ、ない、　、 、こと、もの、、ほう、どちら、これ、にくい、まま、ため、それ、mt、
、てる、com、やる、みたい、つける、ちゃう、あと、なん、たち、もん、ここ、やつ、うち、はず、ドット、コム""".replace('\n', "").split('、')
stops.extend(stopwords)
stops = set(stops)

POS = ['名詞', '動詞', '形容詞', '辞書登録']

In [ ]:
def apply_mecab(df, nayose={}, stops=[], POS=[]):

    def apply_mecab_text(text):

        text = text.strip()    
        t = MeCab.Tagger(r' -u /content/terminology.dic')
        all_words = []
        words = []
        word_list = []
        words = t.parse(text).split('\n')[:-2]
        for w in words:
            parts = w.split('\t')[1].split(',')
            pos = parts[0]
            base = parts[6]
            
            # 数字を入れるための設定
            if base == '*':
                base = w.split('\t')[0]
            
            if pos in POS:
            
                # stopwordsの除去
                if base in stops:
                    base = '' 
                
                # 名寄せ
                if base in nayose.keys():
                    base = nayose[base]

                # 格納
                if len(base) > 1:   
                    word_list.append(base.replace('"', ""))
        
        # 重複単語をなくすための処理
        return sorted(set(word_list), key=word_list.index)


    df_describe = pd.DataFrame(columns=['count', 'mean', 'std', 'min', '25%', '50%', '75%', 'max'])

    name = '形態素解析後'
    df['words'] = df['preprocessed'].apply(apply_mecab_text)
    df_describe.loc[name] = calc_lens(df['words'])
    print(f'{name}: {len(df):,}')  

    # 単語数を数える
    df['n_words'] = df['words'].map(lambda x: len(x))
    # 単語数0のものを削除
    df = df[df['n_words']!=0]

    name = '重複削除後'
    df = df.drop_duplicates(subset='text').reset_index(drop=True)
    df_describe.loc[name] = calc_lens(df['words'])
    print(f'{name}: {len(df):,}')

    display(df_describe)
    return df



In [ ]:
%%time
stops_ = list(stops)
stops_.extend(['若年', '若者'])
yng_df_mecab = apply_mecab(dfs[0], nayose=nayose, stops=stops_, POS=POS)


形態素解析後: 91,023
重複削除後: 90,930


,count,mean,std,min,25%,50%,75%,max
形態素解析後,91023.0,14.31,6.22,0.0,10.0,14.0,19.0,47.0
重複削除後,90930.0,14.33,6.20,1.0,10.0,14.0,19.0,47.0


CPU times: user 1min 11s, sys: 1min, total: 2min 11s
Wall time: 2min 10s


In [ ]:
%%time
stops_ = list(stops)
stops_.extend(['中年', '中高年'])
mid_df_mecab = apply_mecab(dfs[1], nayose=nayose, stops=stops_, POS=POS)

形態素解析後: 22,933
重複削除後: 22,909


,count,mean,std,min,25%,50%,75%,max
形態素解析後,22933.0,14.38,6.49,0.0,9.0,14.0,19.0,41.0
重複削除後,22909.0,14.40,6.48,1.0,9.0,14.0,19.0,41.0


CPU times: user 18.1 s, sys: 15.3 s, total: 33.3 s
Wall time: 33.2 s


In [ ]:
%%time
stops_ = list(stops)
stops_.extend(['高齢', '老人'])
old_df_mecab = apply_mecab(dfs[2], nayose=nayose, stops=stops_, POS=POS)

形態素解析後: 707,109
重複削除後: 703,452


,count,mean,std,min,25%,50%,75%,max
形態素解析後,707109.0,12.64,6.57,0.0,7.0,12.0,17.0,53.0
重複削除後,703452.0,12.70,6.53,1.0,7.0,12.0,18.0,53.0


CPU times: user 8min 40s, sys: 7min 18s, total: 15min 59s
Wall time: 15min 55s


In [ ]:
dfs_mecab = [yng_df_mecab, mid_df_mecab, old_df_mecab]

### 2.4 単語数の絞り込み

In [ ]:
def create_all_dic(df_words):
    all_words= []
    for words in df_words:
        all_words.extend(words)
    all_dic_tupple = Counter(all_words).most_common()
    all_dic = {}
    for x in all_dic_tupple:
        all_dic[x[0]] = x[1]
    return all_dic

def create_all_dic_ratio(all_dic, n):
    for (k,v) in all_dic.items():
        all_dic[k] = round(all_dic[k] / n, 4)
    return all_dic

def reduce_df_words(df, all_dic_ratio, use_ratio=0.001):
    use_words = []
    for (k,v) in all_dic_ratio.items():
        if v >= use_ratio:
            use_words.append(k)
        else:
            break

    print(f'出現率{use_ratio}以上の単語数: {len(use_words)}')
    
    for i, words in tqdm(enumerate(df['words'])):
        reduced_words = []
        for w in words:
            if w in use_words:
                reduced_words.append(w)
            else:
                continue
        df.at[i, 'words'] = reduced_words

    return df    

In [ ]:
%%time
dfs_result = []
for df in dfs_mecab:
    dic_mid = create_all_dic(df['words'])
    all_dic_ratio = create_all_dic_ratio(dic_mid, len(df))
    df= reduce_df_words(df, all_dic_ratio)
    df['words_'] = df['words'].map(lambda x: '_'.join(x))
    df = df.drop_duplicates(subset='words_').reset_index(drop=True)
    df = df.dropna(subset=['words']).reset_index(drop=True)
    df['n_words_'] = df['words'].map(lambda x: len(x))
    df = df[df['n_words_']!=0].reset_index(drop=True)

    dfs_result.append(df)

出現率0.001以上の単語数: 2445


90930it [00:29, 3064.72it/s]


出現率0.001以上の単語数: 2553


22909it [00:08, 2855.94it/s]


出現率0.001以上の単語数: 2084


703452it [02:41, 4366.92it/s]


CPU times: user 3min 25s, sys: 1.45 s, total: 3min 26s
Wall time: 3min 25s


In [ ]:
for df in dfs_result:
    display(df)

,text,time,preprocessed,words,n_words,words_,n_words_
0,若者達は、日本から脱出したほうがいいかもしれない。経歴になんも問題もない奴は、たとえトラック...,2010-01-02,若者達は、日本から脱出したほうがいいかもしれない。経歴になんも問題もない奴は、たとえトラック...,"[日本, 良い, 問題, トラック, 違法, 海外, 仕事, 見つける]",12,日本_良い_問題_トラック_違法_海外_仕事_見つける,8
1,運転してみてわかったのは、横断歩道のスタート地点でないところから道路を横切ることが自分の想像...,2010-01-02,運転してみてわかったのは、横断歩道のスタート地点でないところから道路を横切ることが自分の想像...,"[見る, 横断歩道, スタート, 道路, 横切る, 想像, 危険, 老人, 周り, 平気, ...",13,見る_横断歩道_スタート_道路_横切る_想像_危険_老人_周り_平気_渡る_多い,12
2,「このような、危険な運転をする若者が、後を絶たない」（CV旦那,2010-01-03,このような、危険な運転をする若者が、後を絶たないcv旦那,"[危険, 絶つ, 旦那]",4,危険_絶つ_旦那,3
3,今日夕ご飯王将行ったー名前を書こうとしたらババァに横入りされた。本当オバサンって『最近の若者...,2010-01-04,今日夕ご飯王将行ったー名前を書こうとしたらババァに横入りされた。本当オバサンって最近の若者は...,"[今日, ご飯, 行く, 書く, 入り, 最近, ゆう, 平気, ふざける, 危ない]",13,今日_ご飯_行く_書く_入り_最近_ゆう_平気_ふざける_危ない,10
4,近鉄線の運転手席つき連結部分は若者専用ボックスシート http://twitgoo.com/...,2010-01-04,近鉄線の運転手席つき連結部分は若者専用ボックスシート,"[部分, 専用, シート]",7,部分_専用_シート,3
...,...,...,...,...,...,...,...
86025,そういえば夕方に道走ってたら、一方通行の道なのにおもむろに若者が運転してる車が逆走で侵入して...,2021-12-31,そういえば夕方に道走ってたら、一方通行の道なのにおもむろに若者が運転してる車が逆走で侵入して...,"[言う, 夕方, 走る, 通行, 来る, クラクション, 鳴らす, 慌てる, 付近, 駐車,...",17,言う_夕方_走る_通行_来る_クラクション_鳴らす_慌てる_付近_駐車_後方_信号_変わる,13
86026,若者が｢3時ぐらいに帰るか〜｣って言っててワシ、終電ないやろと思ったけど今日は終夜運転でした,2021-12-31,若者が「3時ぐらいに帰るか」って言っててワシ、終電ないやろと思ったけど今日は終夜運転でした,"[帰る, 言う, ワシ, 今日]",6,帰る_言う_ワシ_今日,4
86027,倍速再生することで今の若者は一体何を得ているのだろう。 感情の経済運転… 感情は効率的に回...,2021-12-31,倍速再生することで今の若者は一体何を得ているのだろう。感情の経済運転…感情は効率的に回すもの...,"[得る, 感情, 経済, 回す, 節約, フル, 思える]",16,得る_感情_経済_回す_節約_フル_思える,7
86028,あけおめウェーイって車道に飛び出してはしゃぐ若者を轢き○さないように安全運転しなきゃ。,2021-12-31,あけおめウェーイって車道に飛び出してはしゃぐ若者を轢き○さないように安全運転しなきゃ。,"[車道, 飛び出す, 轢く, 安全]",7,車道_飛び出す_轢く_安全,4


,text,time,preprocessed,words,n_words,words_,n_words_
0,ドライバーのオッサン、Bollywoodフィルムにでてきそうなハンサム中年。GSで給油中にエ...,2010-01-07,ドライバーのオッサン、bollywoodフィルムにでてきそうなハンサム中年。gsで給油中にエ...,"[オッサン, 来る, エンジン]",8,オッサン_来る_エンジン,3
1,高次脳機能障害のある中年男性。かかりつけ医は眼科。運転免許、復職のための診断書を書く医者を行...,2010-01-09,高次脳機能障害のある中年男性。かかりつけ医は眼科。運転免許、復職のための診断書を書く医者を行...,"[機能, 障害, 男性, 免許, 診断, 書く, 医者, 担当, 職業, 支援, センター,...",20,機能_障害_男性_免許_診断_書く_医者_担当_職業_支援_センター_悩む,12
2,"[tbr] ""かつては「リタイヤしたら田舎で悠々自適」というのが中高年にとって理想の人生では...",2010-01-10,tbrかつてはリタイヤしたら田舎で悠々自適というのが中高年にとって理想の人生ではあった。しか...,"[田舎, 人生, 現実]",13,田舎_人生_現実,3
3,守備範囲は干支一周上まで！ RT @krowed: 中年キラーｗ RT @heidi22...,2010-01-10,守備範囲は干支一周上まで中年キラーw 上目遣いは本当にやめてほしいwwしかも運転席ー助手席の...,"[範囲, やめる, 欲しい, 助手]",10,範囲_やめる_欲しい_助手,4
4,スレッド:97139882 『年寄ドライバーが増えていったら若者や中年は事故回避』 10/0...,2010-01-15,スレッド97139882年寄ドライバーが増えていったら若者や中年は事故回避10 01 15金...,"[増える, 行く, 若者, 事故, 回避, 10, 15, 22, 50]",14,増える_行く_若者_事故_回避_10_15_22_50,9
...,...,...,...,...,...,...,...
21859,歳相応の常識が身につけられてない中年以降はそれなりに多い 圏央道のトンネルで蛇行運転してた...,2021-12-31,歳相応の常識が身につけられてない中年以降はそれなりに多い圏央道のトンネルで蛇行運転してた珍走...,"[常識, 多い, 蛇行, 走る, バイク, 転倒, 後ろ, 乗る, 女性, 仲間, 轢く, ...",19,常識_多い_蛇行_走る_バイク_転倒_後ろ_乗る_女性_仲間_轢く_50代_男性_速報,14
21860,@pivomania お年寄りや杖ついている人や赤子連れを意識するのは良いと思うが、若者や...,2021-12-31,お年寄りや杖ついている人や赤子連れを意識するのは良いと思うが、若者や中年相手まで徹底していた...,"[年寄り, 連れ, 意識, 良い, 若者, 相手, 徹底]",13,年寄り_連れ_意識_良い_若者_相手_徹底,7
21861,中年ペーパードライバーは要練習ということで群馬までドライブです。 https://t.co/...,2021-12-31,中年ペーパードライバーは要練習ということで群馬までドライブです。,"[ペーパー, 練習, 群馬, ドライブ]",4,ペーパー_練習_群馬_ドライブ,4
21862,昭和の高校生が信じていた伝説を中年になって実行する足りなさ。 警察官の目の前でビール 「飲...,2021-12-31,昭和の高校生が信じていた伝説を中年になって実行する足りなさ。警察官の目の前でビール飲酒運転し...,"[昭和, 高校生, 信じる, 警察官, ビール, 飲酒, 逮捕, 放送]",15,昭和_高校生_信じる_警察官_ビール_飲酒_逮捕_放送,8


,text,time,preprocessed,words,n_words,words_,n_words_
0,住基ネット離脱、名古屋市長？という記事がYahooで掲載されているニュースで流れていますが。...,2010-01-01,住基ネット離脱、名古屋市長という記事がyahooで掲載されているニュースで流れていますが。運...,"[ネット, 名古屋, 記事, yahoo, ニュース, 流れる, 免許, 返納, 持つ, 写...",21,ネット_名古屋_記事_yahoo_ニュース_流れる_免許_返納_持つ_写真_付き_カード_機...,17
1,うちの親も含めて、高齢者が運転する車が増えてるだろうなぁ。いままでとは違う世界がもうとっくに...,2010-01-01,うちの親も含めて、高齢者が運転する車が増えてるだろうなぁ。いままでとは違う世界がもうとっくに...,"[含める, 増える, 違う, 世界, 始まる]",5,含める_増える_違う_世界_始まる,5
2,運転免許保有者、約８千万人の中で60歳以上の高齢ドライバーの割合は23％と急増。消費市場にも...,2010-01-01,運転免許保有者、約8千万人の中で60歳以上の高齢ドライバーの割合は23%と急増。消費市場にも...,"[免許, 保有, 60歳, 割合, 23, 急増, 消費, 影響, 与える, マイカー, 乗...",19,免許_保有_60歳_割合_23_急増_消費_影響_与える_マイカー_乗る_向ける_モノ_サー...,17
3,@chacha_oyaji 今、昨年の交通事故死亡者が東京都より埼玉県のほうが多いって報道...,2010-01-02,今、昨年の交通事故死亡者が東京都より埼玉県のほうが多いって報道してましたね。高齢の運転手の危...,"[昨年, 交通事故, 死亡, 東京, 埼玉, 多い, 報道, 危険, 非常, 高い, お気,...",12,昨年_交通事故_死亡_東京_埼玉_多い_報道_危険_非常_高い_お気_付ける,12
4,高齢者の事故が多いのは、高齢者が本来お互いが気をつけなくてはいけない部分をドライバーに委ね過...,2010-01-02,高齢者の事故が多いのは、高齢者が本来お互いが気をつけなくてはいけない部分をドライバーに委ね過...,"[事故, 多い, 本来, お互い, 部分, 過ぎる, 交通事故, 死者]",12,事故_多い_本来_お互い_部分_過ぎる_交通事故_死者,8
...,...,...,...,...,...,...,...
636495,@docmushi タクシーの運転手のおじいちゃんたち、高齢で車高の低い車見落とすかもしれ...,2021-12-31,タクシーの運転手のおじいちゃんたち、高齢で車高の低い車見落とすかもしれないんで仕方ないですね,"[タクシー, おじいちゃん, 低い, 仕方]",5,タクシー_おじいちゃん_低い_仕方,4
636496,2021年の橘華さん 勤務形式が大幅に変わる 高齢者ドライバーに轢かれる ４ヶ月の通院・リ...,2021-12-31,2021年の橘華さん勤務形式が大幅に変わる高齢者ドライバーに轢かれる4ヶ月の通院リハビリ生活...,"[2021, 勤務, 大幅, 変わる, 轢く, 通院, 生活, 制度, 開始, 両親, 旅行...",29,2021_勤務_大幅_変わる_轢く_通院_生活_制度_開始_両親_旅行_大きい_出来事_ほん...,15
636497,｢すごいことしちゃった｣運転手の男は他人事のように･･･ 横浜･4人死傷事故 原因不明のまま...,2021-12-31,「すごいことしちゃった」運転手の男は他人事のように・・・横浜・4人死傷事故原因不明のまま越年...,"[すごい, 人事, 横浜, 死傷, 事故, 原因, 不明, ボケ]",10,すごい_人事_横浜_死傷_事故_原因_不明_ボケ,8
636498,@tiara_music うちも同居してますが両親高齢なんですよね コロナとか車の運転とか...,2021-12-31,うちも同居してますが両親高齢なんですよねコロナとか車の運転とか心配が尽きませんそうですね地味...,"[同居, 両親, コロナ, 心配, 来年, ツイッター, 身体, 下さる, 健康]",13,同居_両親_コロナ_心配_来年_ツイッター_身体_下さる_健康,9


In [ ]:
save_path = 'output/preprocess/'
names = ['yng', 'mid', 'old']

for df, name in zip(dfs, names):
    df = df[['time', 'text', 'preprocessed', 'words']]
    pickle.dump(df, open(f'{save_path}{name}_preprocess.txt', 'wb'))

## 3. 池袋なしデータ
※ 「池袋事故」が高齢運転データに与える影響が大きいため、池袋事故に関連する単語を除いた「池袋なしデータ」を作成した。

In [ ]:
def ikebukuro(df):
    ikebukuro_day = '2019-04-19'
    df_before = df[df['time']<ikebukuro_day]
    df_after = df[df['time']>=ikebukuro_day]

    print('池袋前後データ数')
    a = len(df_before)
    b = len(df_after)
    print(a, b, a+b)

    display(df_before.tail(3))
    display(df_after.head(3))

    ikebukuro_id = []

    for i, words in zip(df_after.index, df_after['words']):
        for w in ['池袋', '上級国民', '飯塚', '幸三']:
            if w in words:
                ikebukuro_id.append(i)
                break
            else:
                continue

    df_ikebukuro = df_after.loc[ikebukuro_id,:]

    other_id = sorted(set(df_after.index) - set(ikebukuro_id))
    df_after_other = df_after.loc[other_id,:]

    print('2019以降の池袋or他のデータ数')
    a = len(df_ikebukuro)
    b = len(df_after_other)
    print(a, b, a+b)

    df_other = pd.concat([df_before, df_after_other])
    
    print()
    print('池袋or他のデータ数')
    a = len(df_ikebukuro)
    b = len(df_other)
    print(a, b, a+b)

    return df_other, df_ikebukuro

In [ ]:
df = dfs[2].copy()
df_other, df_ikebukuro = ikebukuro(df)

池袋前後データ数
310840 396269 707109


,text,time,preprocessed,words,n_words
310837,@mamari_www (´º∀º｀)ｱﾗﾏｧ💧 最近、年配や、老人の方の方が、色々な事で...,2019-04-18,´º∀ºアラマァ最近、年配や、老人の方の方が、色々な事で、マナーなってない様な。車の運転も、...,"[´º∀º, アラマァ, 最近, 年配, 色々, マナー, 若い, 挨拶, 礼儀, 正しい]",10
310838,老人に限らず、明らかにあかんでしょってな運転する人を排除出来ないものだろうか？(´・ω・｀)...,2019-04-18,老人に限らず、明らかにあかんでしょってな運転する人を排除出来ないものだろうか´ω無理筋なのは...,"[限る, 明らか, 排除, 出来る, 無理, 分かる, ジジババ, せい, 事故, 怪我]",10
310839,@Kao19360468 表示していて、使えないなら乗車時に言うべき。基本的に沖縄の高齢タ...,2019-04-18,表示していて、使えないなら乗車時に言うべき。基本的に沖縄の高齢タクシードライバーはクレジット...,"[表示, 使える, 乗車, 言う, 基本, 沖縄, タクシー, クレジットカード, 支払い,...",10


,text,time,preprocessed,words,n_words
310840,@kotatu_pso 止まらない高速道路整備に 増加していく高齢ドライバー。若者はもう車...,2019-04-19,止まらない高速道路整備に増加していく高齢ドライバー。若者はもう車を持たなくなって…本当にどこ...,"[止まる, 高速, 道路, 整備, 増加, 行く, 若者, 持つ, 向かう]",9
310841,RT @MADAYOMEKA: @tachan0379 @AP_koutuu 歩行者と...,2019-04-19,歩行者として道路を渡るシミュレーターを高齢者がやってるのを見ましたが、車が来るのは見えないと...,"[歩行者, 道路, 渡る, シミュレーター, 見る, 来る, 見える, 言う, 距離, つか...",12
310842,@katsuhiko831 無免許の不法外国人労働者とイキリ散らしたガラの悪い輩と暴走族と...,2019-04-19,無免許の不法外国人労働者とイキリ散らしたガラの悪い輩と暴走族と認知症老人。それにα飲酒運転、...,"[免許, 不法, 外国, 労働, イキリ, 散らす, ガラ, 悪い, 暴走, 認知, 飲酒,...",26


2019以降の池袋or他のデータ数
28503 367766 396269

池袋or他のデータ数
28503 678606 707109


In [ ]:
# 比べるもの
# all, 池袋ぬき, 池袋のみ
compare_dfs = [df, df_other, df_ikebukuro]

print([f'{len(df):,}' for df in compare_dfs])

['707,109', '678,606', '28,503']


In [ ]:
df = df_other[['time', 'text', 'preprocessed', 'words']]
name = 'old_exclude_ikebukuro'

pickle.dump(df, open(f'{save_path}{name}_preprocess.txt', 'wb'))